In [1]:
!pip install cvxpy

In [2]:
# !pip install gurobipy

In [4]:
import cvxpy as cp
# import gurobipy as gp
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

In [5]:
np.random.seed(1)
sc=StandardScaler()

In [7]:
df=pd.read_csv('/content/German-Credit.csv')
dataset = df.loc[2:,:]
# dataset.info()
dataset[['Age', 'Duration','Job', 'SavingAccounts','CheckingAccount','CreditAmount']]=dataset[['Age', 'Duration','Job', 'SavingAccounts','CheckingAccount','CreditAmount']].astype('int')
dataset[['Class']]=dataset[['Class']].astype('int')

<ipython-input-7-77f73a055637>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[['Age', 'Duration','Job', 'SavingAccounts','CheckingAccount','CreditAmount']]=dataset[['Age', 'Duration','Job', 'SavingAccounts','CheckingAccount','CreditAmount']].astype('int')
<ipython-input-7-77f73a055637>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[['Class']]=dataset[['Class']].astype('int')


In [14]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 2 to 1001
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Age              1000 non-null   int64 
 1   Sex              1000 non-null   object
 2   Job              1000 non-null   int64 
 3   Housing          1000 non-null   object
 4   SavingAccounts   1000 non-null   int64 
 5   CheckingAccount  1000 non-null   int64 
 6   CreditAmount     1000 non-null   int64 
 7   Duration         1000 non-null   int64 
 8   Purpose          1000 non-null   object
 9   Class            1000 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 78.3+ KB


In [8]:
def preprocess(numerical,categorical):
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    transformations = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical),
            ('cat', categorical_transformer, categorical)])
    return transformations

In [9]:
y = dataset["Class"]
# Split data into train and test
X = dataset.drop("Class", axis=1)

In [10]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=13)

In [11]:
numerical = ['Age', 'Duration','Job', 'SavingAccounts','CheckingAccount','CreditAmount']
categorical = x_train.columns.difference(numerical)
proc=preprocess(numerical,categorical)
x_train=proc.fit_transform(x_train)
x_test=proc.transform(x_test)
y_train=y_train.values
y_test=y_test.values

In [12]:
x_train.shape

(800, 19)

In [13]:
x_train[0]

array([ 0.97905985, -0.24308004,  0.15290124, -0.17465096, -1.03585243,
       -0.45817711,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ])

In [15]:
clf = LogisticRegression(random_state=0).fit(x_train,y_train)

In [16]:
def create_obj_var(x,n_ft,ft_type):
  vars=[]
  obj_var=[0]*n_ft
  for i in range(n_ft):
    # vars[i]=cp.Variable()
    # print(ft_type[i])
    if ft_type[i]=='ct':
      # print(vars[i])
      v=cp.Variable()
      obj_var[i]=(cp.norm(v-x[i],1)) #should add denominator
    else:
      v=cp.Variable(boolean=True)
      obj_var[i]=(cp.maximum(1,(v-x[i])/1000))
    vars.append(v)
  vars=cp.Variable(n_ft)
  return vars,obj_var



In [17]:
# vars,_=create_obj_var(x_test[0],30,['ct']*30)
# vars

In [18]:
# p=vars@clf.coef_[0].T
# cp.exp(p)/1+cp.exp(p)

In [19]:
# np.logaddexp(-1000,x_test[0]@clf.coef_[0]+clf.intercept_[0])

In [20]:
# p=x_test[0]@clf.coef_[0]+clf.intercept_[0]
# (np.exp(p)/(1+np.exp(p)))
# p

In [21]:
def check_pred(cf,clf,target):
  cf=cp.hstack(cf)
  # p=cp.log_sum_exp(cp.hstack([-1000,cf@clf.coef_[0].T+clf.intercept_[0]]))
  p=cp.matmul(cf,clf.coef_[0])+clf.intercept_[0]
  # p=cp.exp(p)/(1+cp.exp(p))
  p=cp.sum(cp.exp(p - cp.log(target)))
  return p



In [22]:
# Create constraint.
def optimize(vars,obj_var,clf,x,target):
  # vars.append(cp.Variable())
  # x=x.reshape(1,-1)

  # p=cp.sum(vars*np.transpose(np.array(clf.coef_))) +clf.intercept_[0]
  # print(p)
  # p=1/1+cp.exp(-p)


  # cf=[]
  # for j in vars:
  #   j.value=np.random.randn(1)[0]

  # v=cp.Variable(30)
  constraints=[check_pred(vars,clf,target)<=1]

  # Form objective.
  objective = cp.Minimize(cp.sqrt(cp.sum_squares(cp.vstack(obj_var)))) #
  # objective = cp.Minimize((cp.sum(cp.vstack(obj_var))))
  # objective = cp.Minimize(cp.norm(v-x,1))

  # vars.value = numpy.random.randn(30)


  # Form and solve problem.
  prob = cp.Problem(objective,constraints)
  prob.solve(solver=cp.ECOS_BB,qcp=True,verbose=True) #cp.SCS, cp.ECOS_BB
  # prob.solve(warm_start=True)
  print("status: {}".format(prob.status))
  return vars

In [ ]:
cf_all=[]
ft_type=['ct']*6+['bb']*13
n_ft=19
for k in range(len(x_test)):
  i=x_test[k]
  target=1-y_test[k]
  vars,obj_var=create_obj_var(i,n_ft,ft_type)
  cf=[]
  target=0.001 if target==0 else 0.999
  vals=optimize(vars,obj_var,clf,i,target)
  for j in vals:
    cf.append(j.value)
  cf_all.append(cf)

In [26]:
np.array(cf_all).shape, y_test.shape

((200, 19), (200,))

In [27]:
validity=0
for i in range(len(cf_all)):
  if clf.predict(np.array(cf_all[i]).reshape(1,-1))==1-y_test[i]:
    validity+=1

print(validity)

142


In [ ]:
def get_stats(x_train,ft_types):
  '''
  should receive the raw data
  '''
  n_ft= x_train.shape[1]
  stats=[]
  for i in range(n_ft):
    s=[]
    if ft_types[i]=='ct':
      s[0]=np.mean(x_train[:,i])
      s[1]=np.var(x_train[:,i])**0.5
    else: #consider only cont and cat currently. Need discrete data also.
      s[0]=np.min(x_train[:,i])
      s[1]=np.max(x_train[:,i])
    stats.append(s)

  return stats


In [ ]:
def gen_random_point(stats,ft_types):
  randx=[]
  for i in range(len(stats)):
    val=0
    if ft_types[i]=='continuous':
      val=np.randn(stats[i][0],stats[i][1],size=None)
    else:
      val=np.random.randint(stats[i][0],stats[i][1])
    randx[i]=val

  return randx

